### Advanced Programming Summative Assessment 

In [1]:
#checking working in Python 3.7
!python -V

Python 3.7.9


In [2]:
#imports

#for GUI
import tkinter as tk
from tkinter import ttk

#for data cleaning and prep
import pandas as pd
import numpy as np

#for conversion to database
import csv
from pathlib import Path
import sqlite3

#for graphs
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import seaborn as sns

### The GUI

In [3]:
#global variable creation
global files
global uk_airports
airports = pd.read_csv('airports.csv')
runways = pd.read_csv('runways.csv')
frequencies = pd.read_csv('airport-frequencies.csv')
files = {'airports.csv':airports, 'runways.csv':runways, 'airport-frequencies.csv':frequencies}

#variation on cleaning and prep function to update global variables
def get_uk_airports(files):
            for key in files.keys():
                file_name = key
                file = files[key]
                #removing duplicates
                duplicates = file.duplicated().sum()
                if duplicates > 0:
                    file.drop_duplicates(inplace=True)
                #drop any rows that are missing everything
                file.dropna(how = 'all') 
            #Remove all rows in airports where ‘type’ = closed
            airports.drop(airports[airports['type'] =='closed'].index, inplace = True)
            #new df of just UK airports 
            uk_airports = airports.loc[airports['iso_region'].str.startswith('GB', na=False)]
            #only keep the rows where type is in small, medium or large
            keep = ['small_airport', 'medium_airport', 'large_airport']
            uk_airports = uk_airports[uk_airports.type.isin(keep)]
            #get_dummies for type
            uk_airports = pd.get_dummies(data=uk_airports, columns=['type'], prefix='Type')
            #joining freq_mhz column to uk_airports
            #renaming id column so it matches
            uk_airports.rename(columns = {'id':'airport_ref'}, inplace = True)
            #merging datasets
            uk_airports = uk_airports.merge(frequencies, on='airport_ref', how='inner') 
            #dropping the unneeded columns from frequencies
            uk_airports.drop(uk_airports.iloc[:, 20:24], inplace = True, axis = 1)
            #adding uk_airports to the files dict
            files['uk'] = uk_airports
            return files, uk_airports

files, uk_airports = get_uk_airports(files)

#GUI creation
class dataConvertor(tk.Tk):
     
    # __init__ function for class
    def __init__(self, *args, **kwargs):
         
        # __init__ function for class Tk
        tk.Tk.__init__(self, *args, **kwargs)
         
        # creating a container
        container = tk.Frame(self) 
        container.pack(side = "top", fill = "both", expand = True)
  
        container.grid_rowconfigure(0, weight = 1)
        container.grid_columnconfigure(0, weight = 1)
  
        # initializing frames to an empty array
        self.frames = {} 
  
        # iterating through tuple of the different page layouts
        for F in (StartPage, Page1, Page2, Page3, Page4):
  
            frame = F(container, self)
  
            # initializing frame of that object from each page
            self.frames[F] = frame
  
            frame.grid(row = 0, column = 0, sticky ="nsew")
  
        self.show_frame(StartPage)
  
    # to display the current frame (passed as param)
    def show_frame(self, cont):
        frame = self.frames[cont]
        frame.tkraise()

#first frame - startpage  
class StartPage(tk.Frame):
    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
         
        # label 1
        label = ttk.Label(self, text ="Welcome to Airports Info!", font = 'none 12 bold').grid(row = 0, column = 2, padx = 10, pady = 10)
        
        #label 2
        label = ttk.Label(self, text ="Do you want to open an exisiting file or load a new file?", font = 'none 10').grid(row = 1, column = 2, padx = 10, pady = 10)
        
        #button 1 (open existing file)
        button1 = ttk.Button(self, text ="Open File", command = lambda : controller.show_frame(Page1)).grid(row = 2, column = 1, padx = 10, pady = 10)
  
        # button 2 (load new file)
        button2 = ttk.Button(self, text ="Load new File",command = lambda : controller.show_frame(Page2)).grid(row = 2, column = 3, padx = 10, pady = 10)


#second frame - page1 (open file page)
class Page1(tk.Frame):
     
    def __init__(self, parent, controller):
         
        tk.Frame.__init__(self, parent)

        #function to open csv file 
        def open_file(file_name, files):
            if file_name in files.keys():
                file = files.get(file_name)
                ttk.Label(self, text='File retrieved').grid(row=6, column=2) 
                return file
            else:
                ttk.Label(self, text='Unknown file, please try again').grid(row=6, column=2)
        
        #function to open database
        def open_db(file_name):
            try:
                con = sqlite3.connect(file_name)
                cur = con.cursor()
                cur.execute('pragma encoding=UTF8')
                ttk.Label(self, text='Database connection made').grid(row=4, column=2) 
            except:
                ttk.Label(self, text='Connection failed - please verify file name and try again').grid(row=4, column=2)
                
        #label
        label = ttk.Label(self, text ="Enter file name you wish to open", font = 'none 12 bold').grid(row = 0, column = 2, padx = 10, pady = 10)
        
        #input
        f = tk.StringVar()
        file_name = ttk.Entry(self, textvariable=f)
        file_name.grid(row=3, column=2)
        
        #output
        result = ttk.Label(self, text='').grid(row=6, column=2)
        
        #button to open csv file
        button3_style = ttk.Style() 
        button3_style.configure('B3.TButton', background='lightblue')
        button3 = ttk.Button(self, text ="Open csv File", style='B3.TButton')
        button3.config(command=lambda: file_name.config(text=open_file(file_name.get(), files)))
        button3.grid(row = 4, column = 1, padx = 10, pady = 10)   

        #button to open db
        button4_style = ttk.Style() 
        button4_style.configure('B4.TButton', background='lightgreen')
        button4 = ttk.Button(self, text ="Open Database", style='B4.TButton')
        button4.config(command=lambda: file_name.config(text=open_db(file_name.get())))
        button4.grid(row = 4, column = 3, padx = 10, pady = 10)          
        
        # button to return to startpage
        button1 = ttk.Button(self, text ="StartPage", command = lambda : controller.show_frame(StartPage)).grid(row = 7, column = 1, padx = 10, pady = 10)
  
        # button to go to outputs
        button2 = ttk.Button(self, text ="Get Outputs", command = lambda : controller.show_frame(Page3)).grid(row = 7, column = 3, padx = 10, pady = 10)

# third frame - page2 (load file)
class Page2(tk.Frame):
    
    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        #label 1
        label1 = ttk.Label(self, text ="Available files : \n\n    airports.csv \n    runways.csv \n    airport-frequencies.csv", font = 'none 12').grid(row = 0, column = 2, padx = 10, pady = 10)
        
        #label 2
        label2 = ttk.Label(self, text ="Do you want to load and clean all these files?", font = 'none 12 bold').grid(row = 1, column = 2, padx = 10, pady = 10)
        
        #function to clean and prep
        def load_clean_and_prep(files):
            
            #cleaning
            for key in files.keys():
                file_name = key
                file = files[key]
                
                #removing duplicates
                duplicates = file.duplicated().sum()
                if duplicates > 0:
                    file.drop_duplicates(inplace=True)
                    
                #drop any rows that are missing everything
                file.dropna(how = 'all') 
                
                #returning files
                print(f'{file_name} loaded and cleaned')

            #preprocessing
            #Remove all rows in airports where ‘type’ = closed
            airports.drop(airports[airports['type'] =='closed'].index, inplace = True)
            
            #new df of just UK airports 
            uk_airports = airports.loc[airports['iso_region'].str.startswith('GB', na=False)]
            
            #only keep the rows where type is in small, medium or large
            keep = ['small_airport', 'medium_airport', 'large_airport']
            uk_airports = uk_airports[uk_airports.type.isin(keep)]
            
            #get_dummies for airport type
            uk_airports = pd.get_dummies(data=uk_airports, columns=['type'], prefix='Type')

            
            #joining freq_mhz column to uk_airports
            #renaming id column so it matches
            uk_airports.rename(columns = {'id':'airport_ref'}, inplace = True)
            #merging datasets
            uk_airports = uk_airports.merge(frequencies, on='airport_ref', how='inner') 
            #dropping the unneeded columns from frequencies
            uk_airports.drop(uk_airports.iloc[:, 20:24], inplace = True, axis = 1)

            #adding uk_airports to the files dict
            files['uk'] = uk_airports
            #output
            ttk.Label(self, text='File formatting completed and uk_airports table created').grid(row=5, column=2) 

        #output
        result = ttk.Label(self, text='')
        result.grid(row=5, column=2) 
        
        #button to load data and produce outputs
        button3 = ttk.Button(self, text="Yes")
        button3.config(command=lambda: result.config(text=load_clean_and_prep(files)))
        button3.grid(row=3, column=1, padx = 10, pady = 10)
        
        # button to return to homepage
        button2 = ttk.Button(self, text ="No, return to Start Page", command = lambda : controller.show_frame(StartPage)).grid(row = 3, column = 3, padx = 10, pady = 10)
        
        # button to translate files to sql
        button1 = ttk.Button(self, text ="Translate files to SQLite", command = lambda : controller.show_frame(Page4)).grid(row = 7, column = 2, padx = 10, pady = 10)   


#fifth frame - translate file page  
class Page4(tk.Frame):
    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        
        #file translation function
        def file_translation(name, files):     
            columns = {'airports.csv': 'id int, ident varchar, type varchar, name varchar, latitude_deg float, longitude_deg float, elevation_ft int, continent varchar, iso_country varchar, iso_region varchar, municipality varchar, scheduled_service boolean, gps_code varchar, iata_code varchar, local_code varchar, home_link varchar, wikipedia_link varchar, keywords varchar',
                      'runways.csv': 'id int, airport_ref int, airport_ident varchar, length_ft int, width_ft int, surface varchar, lighted boolean, closed boolean, le_ident varchar, le_latitude_deg float, le_longitude_deg float, le_elevation_ft int, le_heading_degT int, le_displaced_threshold_ft int, he_ident varchar, he_latitude_deg float, he_longitude_deg float, he_elevation_ft int, he_heading_degT int, he_displaced_threshold_ft int',
                       'airport-frequencies.csv' : 'id int, airport_ref int, airport_ident varchar, type varchar, description varchar, frequency_mhz float',
                      'uk': 'airport_ref int, ident varchar, name varchar, latitude_deg float, longitude_deg float, elevation_ft int, continent varchar, iso_country varchar, iso_region varchar, municipality varchar, scheduled_service boolean, gps_code varchar, iata_code varchar, local_code varchar, home_link varchar, wikipedia_link varchar, keywords varchar,Type_large_airport int,Type_medium_airport int, Type_small_airport int, frequency_mhz float'}   

            #create empty database
            Path(name).touch()
            print(f'Empty database {name} created')
            #create database connection and cursor
            con = sqlite3.connect(name)
            cur = con.cursor()
            cur.execute('pragma encoding=UTF8')
            
            for f in files.keys():
                if f == 'airports.csv':
                    table_name = 'Airports'
                    #get dataset
                    dataset = files[f]
                    #create table
                    cur.execute(f'''CREATE TABLE {table_name} ({columns.get('airports.csv')})''')
                    print('Airports table created')
                    # write the data to the table
                    dataset.to_sql(table_name, con, if_exists='append', index = False)
                    print(f'airports.csv data loaded to Airports table on {name}')
                elif f == 'runways.csv':
                    table_name = 'Runways'
                    #get dataset
                    dataset = files[f]
                    #create table
                    cur.execute(f'''CREATE TABLE {table_name} ({columns.get('runways.csv')})''')
                    print('Runways table created')
                    # write the data to the table
                    dataset.to_sql(table_name, con, if_exists='append', index = False)
                    print(f'runways.csv data loaded to Runways table on {name}')
                elif f == 'airport-frequencies.csv':
                    table_name = 'Frequencies'
                    #get the dataset 
                    dataset = files[f]
                    #create table
                    cur.execute(f'''CREATE TABLE {table_name} ({columns.get('airport-frequencies.csv')})''')
                    print('Frequencies table created')
                    # write the data to the table
                    dataset.to_sql(table_name, con, if_exists='append', index = False)
                    print(f'airport-frequencies.csv data loaded to Frequencies table on {name}')
                elif f =='uk':
                    table_name = 'uk_airports'
                    dataset = files[f]
                    cur.execute(f'''CREATE TABLE {table_name} ({columns.get('uk')})''')
                    print('uk_airports table created')
                    # write the data to the table
                    dataset.to_sql(table_name, con, if_exists='append', index = False)
                    print(f'uk_airports data loaded to uk_airports table on {name}')
                else:
                    print('Error, please try again')
            cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
            ttk.Label(self, text=f'Database creation complete.\nTable names = {cur.fetchall()}').grid(row=6, column=2) 
            
            con.commit()
            con.close()
            
        #label
        label = ttk.Label(self, text ="Please provide the name for the database :", font = 'none 12 bold').grid(row = 0, column = 2, padx = 10, pady = 10)
        
        #input
        file = tk.StringVar()
        name = ttk.Entry(self, textvariable=file)
        name.grid(row=3, column=2)
        
        #translate button
        button1_style = ttk.Style() 
        button1_style.configure('B1.TButton', background='lightblue')
        button1 = ttk.Button(self, text="Create Database", style='B1.TButton')
        button1.config(command=lambda: result.config(text=file_translation(name.get(), files)))
        button1.grid(row=5, column=2, sticky=tk.W, pady=4)
        
        #output
        result = ttk.Label(self, text='')
        result.grid(row=6, column=2) 
        
        # button to return to start
        button2 = ttk.Button(self, text ="Start Page", command = lambda : controller.show_frame(StartPage)).grid(row = 8, column = 1, padx = 10, pady = 10)
        
        # button to go to outputs
        button3 = ttk.Button(self, text ="Outputs", command = lambda : controller.show_frame(Page3)).grid(row = 8, column = 3, padx = 10, pady = 10)

# fourth frame - page3 (outputs)
class Page3(tk.Frame):
    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        label = ttk.Label(self, text ="Select required outputs", font = 'none 12 bold').grid(row = 0, column = 2, padx = 10, pady = 10)
        
        #function for large airport freq data 
        def large_airport_data(uk_airports):
            #making a new dataframe for just large airports
            la_df = uk_airports[uk_airports['Type_large_airport']==1]
            #getting mean, median and mode
            large_airport_mean = la_df['frequency_mhz'].mean()
            large_airport_median = la_df['frequency_mhz'].median()
            large_airport_mode = la_df['frequency_mhz'].mode()[0]
            #returning results
            ttk.Label(self, text=f'''In large UK airports:\nMean frequency used = {format(large_airport_mean, ".2f")} mhz \nMedian frequency used = {large_airport_median} mhz \nMode frequency used = {large_airport_mode} mhz''').grid(row=2, column=1) 
            
        #button for large airport freq data
        button3_style = ttk.Style() 
        button3_style.configure('B3.TButton', background='steelblue')
        button3 = ttk.Button(self, text ="Large Airport Frequencies data", style='B3.TButton')
        button3.config(command = lambda: result.config(text=large_airport_data(uk_airports)))
        button3.grid(row = 1, column = 1, padx = 10, pady = 10)

        #output
        result = ttk.Label(self, text='')
        result.grid(row=2, column=1) 
        
        #function for freq over 100mhz outputs
        def freq_over_100(uk_airports):
            #making a new dataframe for just rows with freq over 100
            over_100 = uk_airports[uk_airports['frequency_mhz']>100]
            #getting mean, median and mode
            freq_over_100_mean = over_100['frequency_mhz'].mean()
            freq_over_100_median = over_100['frequency_mhz'].median()
            freq_over_100_mode = over_100['frequency_mhz'].mode()[0]

            #returning results
            ttk.Label(self, text=f'''In airports with a frequency over 100mhz :\nMean frequency used = {format(freq_over_100_mean, ".2f")} mhz \nMedian frequency used = {freq_over_100_median} mhz \nMode frequency used = {freq_over_100_mode} mhz''').grid(row=2, column=3) 
            
        #button for freq over 100mhz outputs
        button4_style = ttk.Style() 
        button4_style.configure('B4.TButton', background='steelblue')
        button4 = ttk.Button(self, text ="Frequencies over 100mhz data", style='B4.TButton')
        button4.config(command = lambda: result.config(text=freq_over_100(uk_airports)))
        button4.grid(row = 1, column = 3, padx = 10, pady = 10)
        
        #function for small airport freq graph 
        def small_airport_freq(uk_airports):
            #creating df of just small airports
            sa_df = uk_airports[uk_airports['Type_small_airport']==1]
            #getting value_counts
            counts = sa_df.frequency_mhz.value_counts()
            #getting only frequencies used by 2 or more airports
            over_two = counts.loc[counts.values > 2]
            #formating the pie chart
            my_labels = []
            for i in over_two.index:
                my_labels.append(f'{i}mhz')
            my_colours = ['#6c094f', '#fbb162', '#d9cff3', '#eed9e2', '#aedfc0', '#ffc1cc', '#d7f7e2', '#c9d5f8', '#b2beb5', '#d5c9f8', '#bdc29b', '#fff5ee', '#aedfc0']
            
            #creating pop up
            top = tk.Toplevel(self, height=40, width=40)
            top.title("Frequencies used by more than 2 small airports")
            
            #plotting
            fig = Figure(figsize=(6,6), dpi=100) 
            subplot = fig.add_subplot(111)
            subplot.pie(over_two, labels = my_labels, colors=my_colours, autopct='%1.1f%%')
            pie2 = FigureCanvasTkAgg(fig, master=top)
            pie2.get_tk_widget().grid(row=4, column=1)
            
            #informing user 
            ttk.Label(self, text='Pie chart generated of frequencies used in more than two airports').grid(row=4, column=1) 
             
        #button for small aiport freq graph
        button5_style = ttk.Style() 
        button5_style.configure('B5.TButton', background='lightgreen')
        button5 = ttk.Button(self, text ="Small airport frequencies graph", style='B5.TButton')
        button5.config(command = lambda: small_airport_freq(uk_airports))
        button5.grid(row = 3, column = 1, padx = 10, pady = 10)
        
        #function for freq corr vizualisations 
        def freq_corr(uk_airports):
            #making a df just type and freq for heatmap
            corr_df = uk_airports
            corr_df.drop(corr_df.iloc[:, 0:17], inplace = True, axis = 1)

            #first pop up - the distplots
            #making the pop up
            top2 = tk.Toplevel(self, height=40, width=40)
            top2.title("Frequency usage for different airport types")
            #making the graph
            f = plt.Figure(figsize=(8,8))
            ax1 = f.add_subplot(111)
            sns.distplot(corr_df[corr_df['Type_small_airport']==1].frequency_mhz, color="green", hist=False, label="Small airports", ax=ax1)
            sns.distplot(corr_df[corr_df['Type_medium_airport']==1].frequency_mhz, color="blue", hist=False, label="Medium airports", ax=ax1)
            sns.distplot(corr_df[corr_df['Type_large_airport']==1].frequency_mhz, color="red", hist=False, label="Large airports", ax=ax1)
            f.legend()
            #plotting
            canvas = FigureCanvasTkAgg(f, master=top2)
            canvas.get_tk_widget().grid(row=4, column=4)
            
            #second pop up - the heatmap
            #making the popup
            top3 = tk.Toplevel(self, height=40, width=40)
            top3.title("Correlation between airport types and frequency")
            #making the graph
            f = plt.Figure(figsize=(9,9))
            ax2 = f.add_subplot(111)
            sns.heatmap(corr_df.corr(), cmap='Greens', annot=True, ax=ax2)
            #plotting
            canvas = FigureCanvasTkAgg(f, master=top3)
            canvas.get_tk_widget().grid(row=4, column=4)
            
            #informing user
            ttk.Label(self, text='Heatmap and distplots generated of frequency usage and airport types').grid(row=4, column=3) 
            
        #button for freq corr vizualisations
        button6_style = ttk.Style() 
        button6_style.configure('B6.TButton', background='lightgreen')
        button6 = ttk.Button(self, text ="Frequency correlations visualistions", style='B6.TButton')
        button6.config(command = lambda: freq_corr(uk_airports))
        button6.grid(row = 3, column = 3, padx = 10, pady = 10)
        
        # button for opening another file
        button1 = ttk.Button(self, text ="Open Another File", command = lambda : controller.show_frame(Page1)).grid(row = 9, column = 3, padx = 10, pady = 10)
  
        # button to return to start
        button2 = ttk.Button(self, text ="Startpage",command = lambda : controller.show_frame(StartPage)).grid(row = 9, column = 1, padx = 10, pady = 10)
    
# Driver Code
app = dataConvertor()
app.mainloop()

airports.csv loaded and cleaned
runways.csv loaded and cleaned
airport-frequencies.csv loaded and cleaned
uk loaded and cleaned
Empty database example.db created
Airports table created
airports.csv data loaded to Airports table on example.db
Runways table created
runways.csv data loaded to Runways table on example.db
Frequencies table created
airport-frequencies.csv data loaded to Frequencies table on example.db
uk_airports table created
uk_airports data loaded to uk_airports table on example.db


**********

## Additional functions for checking database - not part of main application

In [11]:
#code to get database table data as csv's
#using example.db, an example database I created through GUI,enter name of database you created
db_filename = 'example.db'

def to_csv():
    db = sqlite3.connect(db_filename)
    cursor = db.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    print(f'Tables = {tables}')
    for table_name in tables:
        table_name = table_name[0]
        table = pd.read_sql_query("SELECT * from %s" % table_name, db)
        table.to_csv('Table_' + table_name + '.csv', index_label='index')
    cursor.close()
    db.close()
to_csv()

Tables = [('Airports',), ('Runways',), ('Frequencies',), ('uk_airports',)]


In [13]:
#printing tables and column names to a schema file
db_filename = 'example.db' #enter your database name here
newline_indent = '\n   '

#writing the results to a schema file
with open('schema.txt', 'w') as f:    
    db=sqlite3.connect(db_filename)
    db.text_factory = str
    cur = db.cursor()

    result = cur.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
    table_names = sorted(list(zip(*result))[0])
    print ("\nTables in database are:"+newline_indent+newline_indent.join(table_names), file=f)

    for table_name in table_names:
        result = cur.execute("PRAGMA table_info('%s')" % table_name).fetchall()
        column_names = list(zip(*result))[1]
        print (("\nColumn names for %s:" % table_name)
               +newline_indent
               +(newline_indent.join(column_names)), file=f)

db.close()

In [15]:
#function to test database
db_filename = 'example.db' #enter your database name here

def transaltion_test():
    con = sqlite3.connect(db_filename)
    print('Run a test to see if the data has been loaded successfully')
    table = input('Please provide name of table to check: Airports, Runways or Frequencies')
    column = input('Please provide name of column')
    data = input('Please provide data value to check')
    print(f'Ok so checking table {table} column {column} for a value = to {data}')
    return pd.read_sql(f'''SELECT * FROM {table} WHERE {column} = {data}''', con)
    con.close()
    
transaltion_test()

Run a test to see if the data has been loaded successfully


Please provide name of table to check: Airports, Runways or Frequencies Airports
Please provide name of column id
Please provide data value to check 6527


Ok so checking table Airports column id for a value = to 6527


,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,iso_country,iso_region,municipality,scheduled_service,gps_code,iata_code,local_code,home_link,wikipedia_link,keywords
0,6527,00AZ,small_airport,Cordes Airport,34.305599,-112.165001,3810,None,US,US-AZ,Cordes,no,00AZ,None,00AZ,None,None,None
